In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

In [ ]:
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams['text.usetex'] = False
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
mpl.rcParams['figure.dpi'] = 300

# Regression and Classification

Written by: [Matthew R. Carbone](https://www.bnl.gov/staff/mcarbone) | _Assistant Computational Scientist, Computational Science Initiative, Brookhaven National Laboratory_

In this tutorial, we're going to go over the fundamentals of regression and classification, which are the two most common types of supervised learning. We will also discuss some of the best practices in machine learning, such as a train-validation-testing split. In regression problems, the objective is to learn a _continuous_ output. In classification problems, the objective is to learn a _discrete_ output. Here are some examples:
- Predicting the cost of a house from its properties, such as square footage, number of bathrooms, etc. is a _regression_ problem.
- Whether or not an image is of a cat or dog is a _classification_ problem.
- Predicting the type of animal in an image is a _classification_ problem.

**Learning objectives:**
- Understand a variety of regression and classification algorithms.
- Start to explore some of the fundamental concepts in machine learning, such as splitting data, overfitting, etc.

Regression and classification problems can be solved via a variety of different methods (or _models_). In this tutorial, we're going to go over the following types of models, which will form the backbone of your understanding for e.g. neural networks, and other types of machine learning, later on.
- Linear regression
- Polynomial regression
- Logistic regression

There are numerous [other types of models](https://www.listendata.com/2018/03/regression-analysis.html#Linear-Regression) which we simply won't have the time to dive into, but paradigmatically, the objective of all of these models is the same. Given some input, predict some output.

## Ingredients for regression

There are a few "ingredients" to always consider when approaching a regression problem.
- Your available data ("dataset")
- Your choice of model ("model")
- How you choose to fit the model to the data ("optimizer")
- An indicator for how well your model fits the data ("loss function"/"metric"/"criterion")

We will discuss all of these components today.

## Other resources

- [Andrew Ng's flagship Coursera course on machine learning](https://www.coursera.org/specializations/machine-learning-introduction)
- [Intro to regression analysis](https://towardsdatascience.com/introduction-to-regression-analysis-9151d8ac14b3)
- [15 types of regression](https://www.listendata.com/2018/03/regression-analysis.html#Linear-Regression)
- [Gradient descent tutorial](https://machinelearningmind.com/2019/10/06/gradient-descent-introduction-and-implementation-in-python/)
- [Linear regression gradient descent tutorial](https://towardsdatascience.com/linear-regression-using-gradient-descent-97a6c8700931)

# Linear regression and gradient descent

Let's begin with the simplest form of regression: that of fitting a line to data. We can recast this problem as learning a function $f(x) = y,$ where the form of $f$ is simply the familiar $f(x) = mx + b.$ Given a dataset $\{x_i, y_i\}$, we can "learn" the coefficients $m$ and $b$ that best model the data.

In [ ]:
def linear_model(x, m, b):
    return m * x + b

In [ ]:
def linear_data_with_noise(seed=123, scale=0.5, N=100, slope=2.4, y_intercept=0.8):
    np.random.seed(seed)
    x = np.linspace(-1, 1, N)
    y = linear_model(x, slope, y_intercept) + np.random.normal(scale=scale, size=(N,))
    return x, y

In [ ]:
x, y = linear_data_with_noise()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

ax.scatter(x, y)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")

plt.show()

Suppose we fix $m$ and $b$ to some values $m_0$ and $b_0$. We now have a model which is completely defined, and given some value for $x$, we can predict $y.$ But how do we know if these are good choices? We need to define a metric, or a measure of how well the model fits the data.

In [ ]:
def criterion(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

Above, our `criterion` is called the average mean squared error. The square root of this is the "root-mean-squared" error, which you are likely familiar with. For the purposes of this example, it doesn't really matter.

## Evaluating an arbitrary model using a "random" optimizer

We know what the ground truth slope `m` and y-intercept `b` actually are, but let's pretend we don't, and evaluate a set of linear models against the data that we have. We can take a bunch of values for `m0` (guesses for the slope) and `b0` (guesses for the y-intercept), create a linear model with those parameters, and evaluate those against the actual data.

In [ ]:
np.random.seed(123)
m0 = np.random.random(size=100) * 6 - 3  # 100 random numbers between -3 and 3
b0 = np.random.random(size=100)          # 100 random numbers between 0 and 1

Let's try every one of these combinations for `m0` and `b0`, and valuate the model against our data.

In [ ]:
import pandas as pd
from itertools import product

In [ ]:
m0b0 = list(product(m0, b0))

In [ ]:
df = pd.DataFrame({
    "m0": [params[0] for params in m0b0],
    "b0": [params[1] for params in m0b0],
    "criterion": [criterion(y, linear_model(x, *params)) for params in m0b0]
})

In [ ]:
df

In [ ]:
argmin = df["criterion"].argmin()

In [ ]:
df.iloc[argmin, :]

## ⚠️ Check your understanding/Discussion

What happened here?

## A smarter optimizer: gradient descent

It stands to reason that just randomly guessing parameters is probably not the most effective way of finding the right guess. Can we do better? Absolutely! [Gradient descent](https://en.wikipedia.org/wiki/Gradient_descent), and its family of related methods (you may have heard of [Adam](https://arxiv.org/abs/1412.6980)), can be used to find an optimal set of parameters for arbitrary models.

Gradient descent, and its related methods, are all about finding the optima of functions. For example, if I give you a function $f(x) = (x-2)^2$ and ask you to find its minimum, how would you do this. From calculus, you might know how. You also might recognize this is a simple parabola and know the answer right away. But what about some arbitrary function, possibly with many _local_ minima, and possibly a _huge_ number of parameters (in machine learning, this number can easily be greater than $10^7$)? Gradient descent is a method for finding minima numerically.

Given a function $f(x)$ and its derivative $\nabla f(x),$ the general rule of gradient descent is that to find a point closer to the actual minimum of $f,$ we follow its gradient:

$$x_{n+1} = x_n - \gamma \nabla f(x_n).$$

This is perhaps easiest to see through an example. Let's consider the function $f(x) = (x-2)^2,$ which of course has derivative $df(x)/dx = 2(x-2).$

In [ ]:
def gradient_descent(x0, f_prime, gamma, N=100):
    """The gradient descent algorithm for a scalar function.
    
    Parameters
    ----------
    x0 : float
        The initial guess for the minimum.
    f_prime : callable
        The derivative of the function of interest.
    gamma : float
        Learning rate.
    N : int
        Number of iterations.
        
    Returns
    -------
    list
        A list of the values of x found during gradient descent.
    """
    
    res = [x0]
    for ii in range(N):
        x_n = res[-1]
        res.append(x_n - gamma * f_prime(x_n))
    return res

In [ ]:
points = np.array(gradient_descent(
    x0=10.0,
    f_prime=lambda x: 2.0 * (x - 2.0),
    gamma=0.1,
    N=100
))

We of course know that the true minimum of this function is at $x=2,$ but lets plot how well the `gradient_descent` function finds this minimum.

In [ ]:
cmap = mpl.colormaps["viridis"].resampled(len(points))

fig, ax = plt.subplots(1, 1, figsize=(3, 1))

xgrid = np.linspace(-1, 15, 100)
ax.plot(xgrid, (xgrid - 2.0)**2, "k-", label=r"$f(x)=(x-2)^2$")
ax.scatter(points, (points - 2.0)**2, c=[cmap(ii) for ii in range(len(points))], s=10)
ax.axvline(2.0, zorder=-1, color="red", label="min $f(x)$")
ax.legend(frameon=False, fontsize=5)

ax.set_xlabel("$x$")
ax.set_ylabel("$f(x)$")

plt.show()

## ⚠️ Check your understanding/Discussion

Play around with the above code a bit. Here are some examples of things you can try.
- What happens if you make $\gamma$ really large?
- What happens if you make $\gamma$ really small?
- What happens if you change $x_0$?
- What happens if you change the function? For example, try $f(x) = \sin x$ (remember the derivative from calculus?). How sensitive is this procedure to your choice of $x_0$?

Here's a [video](https://upload.wikimedia.org/wikipedia/commons/transcoded/4/4c/Gradient_Descent_in_2D.webm/Gradient_Descent_in_2D.webm.720p.vp9.webm) of what gradient descent is doing with a lot of points. This arbitrary function is two dimensional. If you are not sure what's happening here, ask!

## Minimizing the loss function

Now we put two pieces together. As a reminder, the criterion, which we will now refer to as a loss function, is a measure of how different two data points are. We wish to minimize the average of this loss function when doing machine learning. 

For example, consider the linear fit from before. Our ground truth "training data" is $\{x_i, y_i\},$ and our model is simply $f(x; m, b) = mx + b$. We want to _minimize_ the difference between the ground truth values and predicted values on our dataset:

$$ L(m, b) = \frac{1}{N} \sum_{i=1}^N (y_i - f(x_i; m, b))^2 $$

Specifically, we want to minimize $L$ with respect to $m$ and $b.$ Stated slightly differently, we want to find the corresponding values of $m, b$ such that $L$ is minimized. We just learned of a way to do this, gradient descent! However, to use gradient descent, we need the partial derivatives of $L$ with respect to all parameters of interest. Luckily, $L$ is differentiable analytically.

NOTE: ALL machine learning models trained with the family of gradient descent methods have to have analytically known derivatives. This includes neural networks!

The derivatives are:

$$ \frac{\partial L(m, b)}{\partial m} = -\frac{2}{N}\sum_{i=1}^N (y_i - f(x_i; m, b)) x_i$$

and

$$ \frac{\partial L(m, b)}{\partial b} = -\frac{2}{N}\sum_{i=1}^N (y_i - f(x_i; m, b)).$$

Following the rule from before, the update rules for $m$ and $b$ are:

$$ m_{n+1} = m_n - \gamma \frac{\partial L(m, b)}{\partial m}$$

and

$$ b_{n+1} = b_n - \gamma \frac{\partial L(m, b)}{\partial b}.$$

In [ ]:
def gradient_descent_linear_model(x, y, m0, b0, gamma=0.1, N=100):
    b = [b0]
    m = [m0]

    for _ in range(N):
        
        y_pred = linear_model(x, m[-1], b[-1])
        
        # Calculate the derivatives of the loss function
        dLdm = -2.0 / len(x) * np.sum(x * (y - y_pred))
        dLdb = -2.0 / len(x) * np.sum(y - y_pred)
        
        # Update
        m.append(m[-1] - gamma * dLdm)
        b.append(b[-1] - gamma * dLdb)

    return m, b

In [ ]:
m, b = gradient_descent_linear_model(x, y, m0=-1.9, b0=-5.0, gamma=0.1, N=100)

If we plot the convergence of `m` and `b` as a function of "epoch" (or number of full passes through the training data) we can see a very fast convergence of the parameters to the true parameters, which are $m = 0.8$ and $b=2.4$:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 1))

ax.axhline(0.8, color="black", linewidth=0.5)
ax.axhline(2.4, color="black", linewidth=0.5)

ax.plot(m, "ro-", linewidth=0, markersize=0.5, label="$m$")
ax.plot(b, "bo-", linewidth=0, markersize=0.5, label="$b$")

ax.legend()

ax.set_xlabel("Epoch")

plt.show()

We can also plot the linear fit itself as a function of epoch.

In [ ]:
cmap = mpl.colormaps["viridis"].resampled(len(m))
grid = np.linspace(-1, 1, 100)

fig, ax = plt.subplots(1, 1, figsize=(3, 2))



for ii, (mm, bb) in enumerate(zip(m, b)):
    ax.plot(grid, linear_model(grid, mm, bb), color=cmap(ii))
    
ax.scatter(x, y, zorder=1)

ax.set_xlabel("$x$")
ax.set_ylabel("$y$")

plt.show()

## ⚠️ Check your understanding/Discussion

As with the previous section, understanding what's going on here is critical. Once again, ask and answer (discuss with your neighbors), what is going on here? **If you can understand this simple example, you will be able to understand all of gradient-based machine learning that we present as part of this tutorial series!**

## California housing dataset

Let's apply the knowledge we've gained from the previous sections on a "real" problem. We'll be using the [California Housing Dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html), which is a classic teaching example in machine learning. We'll also loosely follow along with this [tutorial](https://towardsdatascience.com/introduction-to-regression-analysis-9151d8ac14b3).

In [ ]:
from sklearn.datasets import fetch_california_housing

In [ ]:
data = fetch_california_housing()

# Put the data into a cleaner format:
df = pd.DataFrame({key: data["data"].T[ii] for ii, key in enumerate(data["feature_names"])})
df[data["target_names"][0]] = data["target"]

Before we fit anything, let's do some exploratory data analysis.

In [ ]:
df["MedHouseVal"].max()

In [ ]:
df["MedHouseVal"].min()

In [ ]:
df.corr()

The strongest correlation between our inputs and outputs comes from the `HouseAge`, `AveRooms` and especially `MedInc`. We'll use these three variables as part of a multilinear regression model:

$$ f(x_1, x_2, x_3) = c_0 + c_1 x_1 + c_2 x_2 + c_3 x_3, $$

where $x_1, x_2, x_3$ are the values of the three input variables.

Although there is not much linear correlation between latitude and longitude and our target, we can still make a pretty plot. Kinda looks like California!

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 2))


ax.scatter(df['Latitude'], df['Longitude'], c=df['MedHouseVal'], s=df['Population']/1000)
cbar = fig.colorbar(mpl.cm.ScalarMappable(), ax=ax)
ax.set_xlabel("Latitude")
ax.set_ylabel("Longitude")
cbar.set_label("MedHouseVal")

plt.show()

## ⚠️ Check your understanding/Discussion

Train a multilinear regression model using gradient descent on the California Housing dataset.

## Training and testing

In machine learning, and in any data-driven modeling, we have to "fit" the model on some dataset, but then we want an unbiased estimation of how the model will perform on data it has not yet seen before. We can easily do this with the `sci-kit learn` library.

In [ ]:
X = df[["HouseAge", "AveRooms", "MedInc"]].to_numpy()
y = df["MedHouseVal"].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

Now let's train a linear model. Since you've already done this the hard way, now we can do it the easy way (using existing libraries)! It's also relevant to note that unlike in most machine learning problems (which use non-linear models), linear models can be fit analytically (and quickly).

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.coef_

In [ ]:
model.intercept_

As before, we want to quantify the degree of fit. We can once again use the criterion to determine the quality of the fit. Let's do this on both the training and testing data.

In [ ]:
criterion(y_train, model.predict(X_train))

In [ ]:
criterion(y_test, model.predict(X_test))

Given that the range of the target is roughly 5, this is not a terrible model. We can likely do better by including more variables, or by making our model more complex (which we will explore in future tutorials).

# Logistic Regression